# LLM RAG Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various a RAG system with MLflow.

In [1]:
import os

Set OpenAI Key

In [2]:
os.environ["OPENAI_API_KEY"] = "redacted"

In [3]:
import pandas as pd

import mlflow

## Create a RAG system

Use Langchain and Chroma to create a RAG system that answers questions based on the MLflow documentation.

In [4]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [5]:
loader = WebBaseLoader("https://mlflow.org/docs/latest/index.html")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
)

## Evaluate the RAG system using `mlflow.evaluate()`

Create a simple function that runs each input through the RAG chain

In [6]:
def model(input_df):
    answer = []
    for index, row in input_df.iterrows():
        answer.append(qa(row["questions"]))

    return answer

Create an eval dataset

In [7]:
eval_df = pd.DataFrame(
    {
        "questions": [
            "What is MLflow?",
            "How to run Mlflow.evalaute()?",
            "How to log_table()?",
            "How to load_table()?",
        ],
    }
)

Create a faithfulness metric

In [8]:
from mlflow.metrics import faithfulness, EvaluationExample

# Create a good and bad example for faithfulness in the context of this problem
faithfulness_examples = [
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions. In Databricks, autologging is enabled by default. ",
        score=2,
        justification="The output provides a working solution, using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions.",
        score=5,
        justification="The output provides a solution that is using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
]

faithfulness_metric = faithfulness(model="openai:/gpt-4", examples=faithfulness_examples)
print(faithfulness_metric)

EvaluationMetric(name=faithfulness, greater_is_better=True, long_name=faithfulness, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called faithfulness based on the input and output.
A definition of faithfulness and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Faithfulness is only evaluated with the provided output and provided context, please ignore the provided input entirely when scoring faithfulness. Faithfulne

Create an answer relevance metric

In [9]:
from mlflow.metrics import answer_relevance, EvaluationExample


answer_relevance_metric = answer_relevance(model="openai:/gpt-4")
print(answer_relevance_metric)

EvaluationMetric(name=answer_relevance, greater_is_better=True, long_name=answer_relevance, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_relevance based on the input and output.
A definition of answer_relevance and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Answer relevance measures the appropriateness and applicability of the output with respect to the input. Scores should reflect the extent to 

In [11]:
results = mlflow.evaluate(
    model,
    eval_df,
    model_type="question-answering",
    evaluators="default",
    predictions="result",
    extra_metrics=[faithfulness_metric, answer_relevance_metric, mlflow.metrics.latency()],
    evaluator_config={
        "col_mapping": {
            "inputs": "questions",
            "context": "source_documents",
        }
    },
)
print(results.metrics)

2023/10/25 17:34:41 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/10/25 17:34:41 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Using pad_token, but it is not set yet.


  0%|          | 0/1 [00:00<?, ?it/s]

2023/10/25 17:35:44 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/10/25 17:35:44 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/10/25 17:35:45 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: perplexity
Using pad_token, but it is not set yet.


  0%|          | 0/1 [00:00<?, ?it/s]

2023/10/25 17:35:48 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/10/25 17:35:48 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/10/25 17:35:48 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2023/10/25 17:35:48 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: faithfulness
2023/10/25 17:35:48 WARNING mlflow.openai.api_request_parallel_processor: Encountered rate limit error. Pausing to cool down for 14.99974799156189 seconds...
2023/10/25 17:36:03 WARNING mlflow.openai.api_request_parallel_processor: Encountered rate limit error. Pausing to cool down for 0.0867919921875 seconds...
2023/10/25 17:36:09 WARNING mlflow.openai.api_request_parallel_processor: 2 rate limit errors received. Consider running at a lower rate.
2023/10/25 17:36:09 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: answer_rele

{'toxicity/v1/mean': 0.00018985224596690387, 'toxicity/v1/variance': 2.445330018924171e-09, 'toxicity/v1/p90': 0.0002462980875861831, 'toxicity/v1/ratio': 0.0, 'perplexity/v1/mean': 66.7759838104248, 'perplexity/v1/variance': 5395.569382352918, 'perplexity/v1/p90': 144.32936973571782, 'flesch_kincaid_grade_level/v1/mean': 8.0, 'flesch_kincaid_grade_level/v1/variance': 21.705000000000002, 'flesch_kincaid_grade_level/v1/p90': 12.910000000000002, 'ari_grade_level/v1/mean': 9.55, 'ari_grade_level/v1/variance': 29.762499999999992, 'ari_grade_level/v1/p90': 15.540000000000001, 'faithfulness/v1/mean': 2.75, 'faithfulness/v1/variance': 3.1875, 'faithfulness/v1/p90': 4.7, 'answer_relevance/v1/mean': 5.0, 'answer_relevance/v1/variance': 0.0, 'answer_relevance/v1/p90': 5.0}


In [12]:
results.tables["eval_results_table"]

questions  \
0                What is MLflow?   
1  How to run Mlflow.evalaute()?   
2            How to log_table()?   
3           How to load_table()?   

                                             outputs  \
0   MLflow is an open source platform for managin...   
1   You can use the Mlflow.evaluate() function to...   
2   log_table() is a function that is part of the...   
3          load_table() is not a function in MLflow.   

                           query  \
0                What is MLflow?   
1  How to run Mlflow.evalaute()?   
2            How to log_table()?   
3           How to load_table()?   

                                    source_documents    latency  token_count  \
0  [{'lc_attributes': {}, 'lc_namespace': ['langc...  42.665819          176   
1  [{'lc_attributes': {}, 'lc_namespace': ['langc...   1.288865           48   
2  [{'lc_attributes': {}, 'lc_namespace': ['langc...   1.289071           47   
3  [{'lc_attributes': {}, 'lc_namespace': ['langc...   3.670376           11   

   toxicity/v1/score  perplexity/v1/score  \
0           0.000208            28.626591   
1           0.000263            21.149670   
2           0.000145            23.411400   
3           0.000144           193.916275   

   flesch_kincaid_grade_level/v1/score  ari_grade_level/v1/score  \
0                                 15.4                      18.9   
1                                  7.0                       6.0   
2                                  7.1                       7.7   
3                                  2.5                       5.6   

   faithfulness/v1/score                      faithfulness/v1/justification  \
0                      5  The output provided by the model is completely...   
1                      1  The output claims that there is a function cal...   
2                      4  The output claims that log_table() is a functi...   
3                      1  The output claims that "load_table() is not a ...   

   answer_relevance/v1/score  \
0                          5   
1                          5   
2                          5   
3                          5   

                   answer_relevance/v1/justification  
0  The output provided by the model is highly rel...  
1  The output provided by the model directly addr...  
2  The output provided by the model directly addr...  
3  The output directly addresses the input questi...